## 通过神经网络预测房价
在这个项目中，我们希望能够构建神经网络来预测房屋的价格

首先，我们导入一些必要的库

In [ ]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

读取训练集和测试集的数据

In [ ]:
train = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('./dataset/test.csv')

可以具体看看前面 5 个训练集长什么样子，可以看到，前面都是这个房屋的属性，最后是房屋的价格

In [ ]:
train.head()

接着我们可以看看训练集和测试集分别有多少个样本

In [ ]:
print('一共有 {} 个训练集样本'.format(train.shape[0]))
print('一共有 {} 个测试集样本'.format(test.shape[0]))

接着我们开始对数据进行处理，首先我们取出**第二个特征**到**倒数第二个特征**，这些特征作为我们神经网络的输入特征

In [ ]:
all_features = pd.concat((train.loc[:, 'MSSubClass':'SaleCondition'],
                          test.loc[:, 'MSSubClass':'SaleCondition']))

接着我们需要进行数据标准化，对于所有的数值特征，我们都会减去均值，除以方差

In [ ]:
numeric_feats = all_features.dtypes[all_features.dtypes != "object"].index # 取出所有的数值特征

# 减去均值，除以方差
all_features[numeric_feats] = all_features[numeric_feats].apply(lambda x: (x - x.mean()) 
                                                                / (x.std()))

如果你仔细看看上面的特征，你会发现，除了数值特征之外，还有很多非数值特征，这些特征我们没有办法将其转换成数值表示，所以我们通过 pandas 的内置函数将其转换成种类表示

比如 **MSZoning** 有两种可能，一种是 RL，一种是 RM，那么我们就将这个特征变成两个新的特征，RL 和 RM，如果这个数据在 **MSZoning** 上是 RL，那么 RL 取 1，RM 取 0；反之如果这个特征是 RM，那么 RL 取 0，RM 取 1.

| RL | RM |
|-|-|
| 0 | 1 |
| 1 | 0 |

In [ ]:
all_features = pd.get_dummies(all_features, dummy_na=True)

除此之外，我们会发现整个数据中有一些丢失数据，这些丢失数据都是 'NA'，我们没有办法将这些数据输入到网络中，所以需要对这些丢失数据进行赋值，这里我们将数据的均值填入到丢失数据中

In [ ]:
all_features = all_features.fillna(all_features.mean())

前面我们已经做好了数据的预处理，下面我们将所有的训练集和验证集都取出成为一个 numpy 的数组

In [ ]:
num_train = train.shape[0]

train_features = all_features[:num_train].as_matrix().astype(np.float32)
test_features = all_features[num_train:].as_matrix().astype(np.float32)

train_labels = train.SalePrice.as_matrix()[:, None].astype(np.float32)
test_labels = test.SalePrice.as_matrix()[:, None].astype(np.float32)

下面需要你来填写代码完成本次的项目

**注意：你只需要完成下面 todo 的部分**

In [ ]:
import tensorflow as tf
import tensorflow.contrib.slim as slim

现在我们需要构造容纳输入和输出的占位符, 之后我们一直填入元素即可

In [ ]:
input_ph = tf.placeholder(tf.float32, (None, 331), name='input_ph')

In [ ]:
#TODO
# 构造接收 label 的占位符

label_ph = None

### 构造神经网络

接下来我们开始构造神经网络, 可以像课程中构造一些基本的单元快速搭建(例如我们用过`from nets import DNN`), 

也可以试着自己独立构造符合自己使用习惯的函数

In [ ]:
def model(inputs, scope='model', reuse=None):
    '''构造模型函数
    
    参数:
      inputs: 2维输入tensor.
      scope: 默认参数域.
      reuse: 是否重用参数域内的参数
      
    返回:
      net: 网络输出
    '''
    with tf.variable_scope(scope, reuse=reuse):
        #TODO
        # 构造神经网络
        net = None
        
        return net

In [ ]:
with slim.arg_scope([slim.fully_connected], 
                    activation_fn=tf.nn.relu, 
                    weights_regularizer=slim.regularizers.l2_regularizer(0.0005)):
    y_ = model(input_ph)

In [ ]:
# 可以调整的超参

batch_size = 10
epochs = 100
use_gpu = False
lr = 0.1
weight_decay = 10

### 使用 mse 作为 loss 函数

使用 Tensorflow 可以从最基本的 op 中定义 mse, 也可以调用`tf.losses`模块中预先定义好的函数

In [ ]:
#TODO
# 构造 mse 形式的 loss 函数
mse = None

### 构造优化方法
我们需要定义模型参数的更新规则, 也就是优化方法

In [ ]:
#TODO
# 生成一个优化器, 可以从 tf.train 里面寻找
opt = None

In [ ]:
#TODO
# 使用优化器生成一个训练 op
train_op = None

在这里, 我们定义一个数据迭代器, 帮助训练

In [ ]:
import random

def get_data(x, y, batch_size=50, shuffle=False):
    indices = range(x.shape[0])
    
    if shuffle:
        random.shuffle(indices)
    
    ind = 0
    while (ind + batch_size <= x.shape[0]):
        batch_x = x[indices[ind: ind + batch_size], :]
        batch_y = y[indices[ind: ind + batch_size], :]
        ind += batch_size
        
        yield batch_x, batch_y
        
    if ind < x.shape[0]:
        batch_x = x[ind:, :]
        batch_y = y[ind:, :]
        
        yield batch_x, batch_y
    else:
        raise StopIteration

在评估模型的时候，为了保证大的价格和小的价格对模型都有着近似相同的影响，我们不会直接使用前面定义的均方误差作为最后的评价函数，我们会对预测的价格和真实的价格取 log，然后计算他们之间均方误差的平方根来作为评价指标，这里的指标我们已经在 `utils.py` 中实现了，感兴趣的同学可以去看看。

In [ ]:
from utils import get_rmse_log

### 构造训练函数

接下来将下面函数中 **TODO** 的部分完成

In [ ]:
def train_model(input_ph, label_ph, pred_op, train_op, 
                    x_train, y_train, x_valid, y_valid, epochs):
    metric_log = dict()
    metric_log['train_loss'] = list()
    if x_valid is not None:
        metric_log['valid_loss'] = list()
    
    #TODO
    # 开启一个 Session
    sess = None
    
    #TODO
    # 对所有参数进行初始化
    pass
    
    for e in range(epochs):
        train_data = get_data(x_train, y_train, batch_size, True)
        if x_valid is not None:
            valid_data = get_data(x_valid, y_valid, batch_size, False)
        else:
            valid_data = None
            
        # 训练模型
        for data in train_data:
            x, y = data
            #TODO
            # 使用 session 运行 train_op
            pass
            
        # 测试训练集
        train_data = get_data(x_train, y_train, batch_size, False)
        pred_train = list()
        for x, _ in train_data:
            #TODO
            # 使用 session 运行 pred_op
            pred = None
            pred_train.append(pred)
        
        pred_train = np.concatenate(pred_train, axis=0)
        metric_log['train_loss'].append(get_rmse_log(pred_train, y_train))
        
        # 测试验证集
        if x_valid is not None:
            valid_data = get_data(x_valid, y_valid, batch_size, False)
            pred_valid = list()
            for x, _ in valid_data:
                #TODO
                # 使用 session 运行 pred_op
                pred = None
                pred_valid.append(pred)

            pred_valid = np.concatenate(pred_valid, axis=0)
            metric_log['valid_loss'].append(get_rmse_log(pred_valid, y_valid))
            
            print_str = 'epoch: {}, train loss: {:.3f}, valid loss: {:.3f}'\
            .format(e+1, metric_log['train_loss'][-1], metric_log['valid_loss'][-1])
        else:
            print_str = 'epoch: {}, train loss: {:.3f}'.format(e+1, metric_log['train_loss'][-1])
        if (e + 1) % 10 == 0:
            print(print_str)
            print()
            
    sess.close()

    # =======不要修改这里的内容========
    # 可视化
    figsize = (10, 5)
    fig = plt.figure(figsize=figsize)
    plt.plot(metric_log['train_loss'], color='red', label='train')
    if valid_data is not None:
        plt.plot(metric_log['valid_loss'], color='blue', label='valid')
    plt.legend(loc='best')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.show()
    
    return metric_log

In [ ]:
metric_log = train_model(input_ph, label_ph, y_, train_op, train_features, train_labels, test_features, test_labels, epochs=epochs)